# **Import Libraries**

In [37]:
import mne

# Utility
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import os

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm

from utilities import read_xdf, epoching
from models import CNNModel

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import confusion_matrix, classification_report

np.random.seed(42)

# **Load Data**

In [38]:
annotations_des = {
    '1': 'Left cue start',
    '2': 'Left stimuli start',
    '3': 'Left blank start',
    '4': 'Right cue start',
    '5': 'Right stimuli start',
    '6': 'Right blank start',
}

In [39]:
def get_epoch(filenames):
    # Define temporal variables
    X, F, t, y = [],[],[],[]
    for filename in filenames:
        ##--------Get First 5 second--------##
        raw = read_xdf(filename, show_plot=False, show_psd=False, verbose=False)
        epochs = epoching(raw, show_psd=False,
            show_eeg=False,  # set True if wanna see preview of all epoch
            tmax=5)         # tmax=5 means set epoch duration 5 second
        # Pick only event 2: Left stimuli, 5: Right stimuli
        epochs = epochs['2','5']
        X.append((  epochs.get_data() * 1e6)[:,:,:1250]) # select only first 1250 frames(approx 5 second)
        F.append(   epochs.compute_psd(method='welch', fmax=30).get_data())
        t.append((  epochs.times)[:1250])                # select only first 1250 frames(approx 5 second)
        y.append(   epochs.events[:, -1])
        ##--------Get Last 5 second--------##
        epochs = epoching(raw, show_psd=False,
            show_eeg=False,  # set True if wanna see preview of all epoch
            baseline=(5,10),
            tmin=5,
            tmax=10)         # tmax=5 means set epoch duration 5 second
        # Pick only event 2: Left stimuli, 5: Right stimuli
        epochs = epochs['2','5']
        X.append((  epochs.get_data() * 1e6)[:,:,:1250]) # select only first 1250 frames(approx 5 second)
        F.append(   epochs.compute_psd(method='welch', fmax=30).get_data())
        t.append((  epochs.times)[:1250])                # select only first 1250 frames(approx 5 second)
        y.append(   epochs.events[:, -1])
    # Concatenate all data
    X = np.concatenate(X)
    F = np.concatenate(F)
    t = np.concatenate(t)
    y = np.concatenate(y)
    return X, F, t, y

## **Load Left-Right Data**

In [40]:
filenames = {'data/Pipo_1_5_test1.xdf'}

Xlr, Flr, tlr, ylr = get_epoch(filenames)

Creating RawArray with float64 data, n_channels=8, n_times=49650
    Range : 0 ... 49649 =      0.000 ...   198.521 secs
Ready.
Used Annotations descriptions: ['1', '2', '3', '4', '5', '6']
Not setting metadata
48 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 48 events and 1251 original time points ...
1 bad epochs dropped
Effective window size : 5.002 (s)
Used Annotations descriptions: ['1', '2', '3', '4', '5', '6']
Not setting metadata
48 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 48 events and 1252 original time points ...
1 bad epochs dropped
Effective window size : 5.006 (s)


In [41]:
Xlr.shape, Flr.shape, tlr.shape, ylr.shape

((32, 5, 1250), (32, 5, 151), (2500,), (32,))

In [42]:
# Map label to 0,1
ylr = np.where(ylr==2, 0, 1)
ylr

array([1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0])

## **Load Free-State Data**

In [43]:
filenames = {'data/Pipo_1_5_test1.xdf'}

Xfs, Ffs, tfs, yfs = get_epoch(filenames)

Creating RawArray with float64 data, n_channels=8, n_times=49650
    Range : 0 ... 49649 =      0.000 ...   198.521 secs
Ready.
Used Annotations descriptions: ['1', '2', '3', '4', '5', '6']
Not setting metadata
48 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 48 events and 1251 original time points ...
1 bad epochs dropped
Effective window size : 5.002 (s)
Used Annotations descriptions: ['1', '2', '3', '4', '5', '6']
Not setting metadata
48 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 48 events and 1252 original time points ...
1 bad epochs dropped
Effective window size : 5.006 (s)


In [44]:
# Map label of y to 2
yfs[yfs==5] = 2
yfs

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [45]:
Xfs.shape, Ffs.shape, tfs.shape, yfs.shape

((32, 5, 1250), (32, 5, 151), (2500,), (32,))

## **Concatenate Data**

In [28]:
X = np.concatenate([Xlr, Xfs])
F = np.concatenate([Flr, Ffs])
t = np.concatenate([tlr, tfs])
y = np.concatenate([ylr, yfs])

X.shape, F.shape, t.shape, y.shape

((64, 5, 1250), (64, 5, 151), (5000,), (64,))

# **Preprocessing**

In [29]:
enc = OneHotEncoder()
Y = enc.fit_transform(y.reshape(-1, 1)).toarray()

In [30]:
X.shape, F.shape, t.shape, Y.shape

((64, 5, 1250), (64, 5, 151), (5000,), (64, 3))

## **Training pipeline**

### Train Test Split

In [31]:
X_train_CNN, X_test_CNN, y_train_CNN, y_test_CNN = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train_CNN = tf.convert_to_tensor(X_train_CNN, dtype=tf.float32)
X_test_CNN = tf.convert_to_tensor(X_test_CNN, dtype=tf.float32)
y_train_CNN = tf.convert_to_tensor(y_train_CNN, dtype=tf.int32)
y_test_CNN = tf.convert_to_tensor(y_test_CNN, dtype=tf.int32)

# **Model Pipeline**

## **Load Model**

In [32]:
CNN = CNNModel()
CNN.model.summary()

Model: "CNNModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 5, 1250)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 3, 128)            480128    
                                                                 
 batch_norm_layer1 (BatchNor  (None, 3, 128)           512       
 malization)                                                     
                                                                 
 maxpool_layer1 (MaxPooling1  (None, 1, 128)           0         
 D)                                                              
                                                                 
 dropout_layer1 (Dropout)    (None, 1, 128)            0         
                                                                 
 dense_layer1 (Dense)        (None, 1, 64)             825

## **Train Model**

In [33]:
CNN.model_train(X_train_CNN, y_train_CNN)
# CNN.load_weights('CNN.h5')

Epoch 1/5
13/13 [==============================] - 1s 21ms/step - loss: 1.2308 - accuracy: 0.4118
Epoch 2/5
13/13 [==============================] - 0s 11ms/step - loss: 1.2237 - accuracy: 0.3725
Epoch 3/5
13/13 [==============================] - 0s 11ms/step - loss: 1.1725 - accuracy: 0.5098
Epoch 4/5
13/13 [==============================] - 0s 10ms/step - loss: 1.1738 - accuracy: 0.2941
Epoch 5/5
13/13 [==============================] - 0s 10ms/step - loss: 0.8758 - accuracy: 0.6078


## **Evaluate the model**

In [34]:
predictions = CNN.model_predict_classes(X_test_CNN)
predictions

1/1 [==============================] - 0s 99ms/step


array([0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2], dtype=int64)

In [35]:
# # Classification report
print(classification_report(np.argmax(y_test_CNN, axis=1), predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         6
           2       0.43      0.43      0.43         7

    accuracy                           0.23        13
   macro avg       0.14      0.14      0.14        13
weighted avg       0.23      0.23      0.23        13



## **Save Model**

In [36]:
# CNN.model.save('CNNNorm.h5')